# Общие внутренние государственные расходы на здравоохранение в процентах от валового внутреннего продукта (ВВП) (%)

In [1]:
import pandas as pd

from helpers import get_data_with_full_locations_and_years

from warnings import simplefilter
simplefilter('ignore')

Эти данные можно найти на сайте ВОЗ по ссылке

https://www.who.int/data/gho/data/indicators/indicator-details/GHO/domestic-general-government-health-expenditure-(gghe-d)-as-percentage-of-gross-domestic-product-(gdp)-(-)

In [2]:
# Загрузим таблицу
data = pd.read_csv('../data/who_health_care_costs_gdp.csv')

data.head(2)

,IndicatorCode,Indicator,ValueType,ParentLocationCode,ParentLocation,Location type,SpatialDimValueCode,Location,Period type,Period,...,FactValueUoM,FactValueNumericLowPrefix,FactValueNumericLow,FactValueNumericHighPrefix,FactValueNumericHigh,Value,FactValueTranslationID,FactComments,Language,DateModified
0,GHED_GGHE-DGDP_SHA2011,Domestic general government health expenditure...,numeric,EMR,Eastern Mediterranean,Country,SOM,Somalia,Year,2020,...,NaN,NaN,NaN,NaN,NaN,0.15,NaN,NaN,EN,2024-12-09T21:00:00.000Z
1,GHED_GGHE-DGDP_SHA2011,Domestic general government health expenditure...,numeric,EMR,Eastern Mediterranean,Country,YEM,Yemen,Year,2020,...,NaN,NaN,NaN,NaN,NaN,0.39,NaN,NaN,EN,2024-12-09T21:00:00.000Z


Описание нужных нам столбцов.

- `ParentLocationCode` - код региона
- `SpatialDimValueCode` - код страны
- `Period` - год
- `FactValueNumeric` - текущие расходы на здравоохранение (CHE) в процентах от валового внутреннего продукта (ВВП) (%)

Оставим только их.

In [3]:
data = data[[
    'ParentLocationCode', 'SpatialDimValueCode', 'Period', 'FactValueNumeric',
]]

data.rename(columns={'FactValueNumeric': 'HealthCareCostsGdp'}, inplace=True)

data.head()

,ParentLocationCode,SpatialDimValueCode,Period,HealthCareCostsGdp
0,EMR,SOM,2020,0.15
1,EMR,YEM,2020,0.39
2,SEAR,BGD,2020,0.41
3,AMR,HTI,2020,0.41
4,AFR,NGA,2020,0.51


В таблице для некоторых стран данные заполнены не за все года.\
Добавим строки с отсутствующими годами, чтобы явно увидеть пропуски.

In [4]:
data = get_data_with_full_locations_and_years(
    data,
    {
        'region_code': 'ParentLocationCode',
        'location_code': 'SpatialDimValueCode',
        'year': 'Period',
    }
)

# Посмотрим на пропуски
data.isna().sum()

ParentLocationCode      0
SpatialDimValueCode     0
Period                  0
HealthCareCostsGdp     77
dtype: int64

Заполним пропуски в пропущенные года средним по региону.

In [5]:
data['HealthCareCostsGdp'] = \
    data.groupby(['ParentLocationCode', 'Period'])['HealthCareCostsGdp'] \
    .transform(lambda x: x.fillna(x.mean()));

# Убедимся, что пропусков нет
data.isna().sum().sum()

0

In [6]:
# Удалим столбцы, которые больше не нужны
data.drop(columns=['ParentLocationCode'], inplace=True)

In [7]:
# Сохраним полученную таблицу для дальнейшего использования
data.to_csv('../data/who_health_care_costs_gdp_prepared.csv', index=False)